<center>
 <h3>Initial OLS Regression for Exploratory Data Analysis </h3>
<center>

In [8]:
import pandas as pd
import statsmodels.formula.api as smf

# Load the data
data = pd.read_csv('ATM_sample.csv')

# Fit the OLS regression model using a formula
ols_model = smf.ols(formula='Withdraw ~ Shops + ATMs + Downtown + Workday + Center + High', data=data).fit()

# Print the summary of the OLS regression
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:               Withdraw   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.656e+05
Date:                Fri, 08 Nov 2024   Prob (F-statistic):               0.00
Time:                        20:58:38   Log-Likelihood:                -51380.
No. Observations:               22000   AIC:                         1.028e+05
Df Residuals:                   21993   BIC:                         1.028e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.4284      0.111     94.198      0.0

<center>
<h3> Ridge Regression model implementation </h3>
<center>

<center>Test Ridge configuration to see which model will be the best option <center>
<center>Finding output: 'Model': 'Polynomial Degree 3 + Interactions' <center>

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

# Load Data
data = pd.read_csv('ATM_sample.csv')
X = data[['Shops', 'ATMs', 'Downtown', 'Workday', 'Center', 'High']]
y = data['Withdraw']

# Split Data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Model Configurations
configs = [
    {'name': 'Base Features', 'poly_degree': 1, 'interactions': False},
    {'name': 'With Interactions', 'poly_degree': 1, 'interactions': True},
    {'name': 'Polynomial Degree 2', 'poly_degree': 2, 'interactions': False},
    {'name': 'Polynomial Degree 2 + Interactions', 'poly_degree': 2, 'interactions': True},
    {'name': 'Polynomial Degree 3', 'poly_degree': 3, 'interactions': False},
    {'name': 'Polynomial Degree 3 + Interactions', 'poly_degree': 3, 'interactions': True}
]

alphas = np.logspace(-4, 4, 50)
results = []

def add_polynomial_terms(X, degree):
    X_poly = X.copy()
    if degree >= 2:
        X_poly['Shops^2'] = X['Shops'] ** 2
        X_poly['ATMs^2'] = X['ATMs'] ** 2
    if degree >= 3:
        X_poly['Shops^3'] = X['Shops'] ** 3
        X_poly['ATMs^3'] = X['ATMs'] ** 3
    return X_poly

# Iterate over configurations
for config in configs:
    if config['interactions']:
        poly = PolynomialFeatures(degree=config['poly_degree'], include_bias=False)
        X_train_poly = poly.fit_transform(X_train)
        X_val_poly = poly.transform(X_val)
    else:
        X_train_poly = add_polynomial_terms(X_train, config['poly_degree'])
        X_val_poly = add_polynomial_terms(X_val, config['poly_degree'])

    scaler = StandardScaler()
    X_train_poly_scaled = scaler.fit_transform(X_train_poly)
    X_val_poly_scaled = scaler.transform(X_val_poly)

    ridge = Ridge()
    param_grid = {'alpha': alphas}
    grid_search = GridSearchCV(ridge, param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train_poly_scaled, y_train)

    best_alpha = grid_search.best_params_['alpha']
    best_mse_cv = -grid_search.best_score_
    best_model = grid_search.best_estimator_

    y_val_pred = best_model.predict(X_val_poly_scaled)
    val_mse = mean_squared_error(y_val, y_val_pred)

    results.append({
        'Model': config['name'],
        'Poly Degree': config['poly_degree'],
        'Interactions': config['interactions'],
        'Best Alpha': best_alpha,
        'MSE (CV)': best_mse_cv,
        'Validation MSE': val_mse
    })

# Display Results
print("\nModel Performance Summary:")
for result in results:
    print(result)

# Save best configuration for Part 2
best_config = min(results, key=lambda x: x['Validation MSE'])
print("\nBest Model Configuration:", best_config)


Model Performance Summary:
{'Model': 'Base Features', 'Poly Degree': 1, 'Interactions': False, 'Best Alpha': 0.009102981779915217, 'MSE (CV)': 6.277284680511846, 'Validation MSE': 6.202448279230941}
{'Model': 'With Interactions', 'Poly Degree': 1, 'Interactions': True, 'Best Alpha': 0.009102981779915217, 'MSE (CV)': 6.277284680511798, 'Validation MSE': 6.202448279231076}
{'Model': 'Polynomial Degree 2', 'Poly Degree': 2, 'Interactions': False, 'Best Alpha': 0.040949150623804234, 'MSE (CV)': 6.22156500390486, 'Validation MSE': 6.127093280965384}
{'Model': 'Polynomial Degree 2 + Interactions', 'Poly Degree': 2, 'Interactions': True, 'Best Alpha': 0.05963623316594643, 'MSE (CV)': 1.5560108496512348, 'Validation MSE': 1.6408938452361017}
{'Model': 'Polynomial Degree 3', 'Poly Degree': 3, 'Interactions': False, 'Best Alpha': 0.0029470517025518097, 'MSE (CV)': 6.221782268707377, 'Validation MSE': 6.127521167198706}
{'Model': 'Polynomial Degree 3 + Interactions', 'Poly Degree': 3, 'Interacti

<center>Apply Ridge Regularization on entire dataset <center>
<center>Model: Polynomial Degree 3 (for ATMs and Shops) + Interactions<center>
<center>Best alpha: 0.019306977288832496<center>

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import cross_val_score

# Load Data 
data = pd.read_csv('ATM_sample.csv')
X = data[['Shops', 'ATMs', 'Downtown', 'Workday', 'Center', 'High']]
y = data['Withdraw']

poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)
feature_names = poly.get_feature_names_out(X.columns)

# Function to check if a feature is valid (no higher powers or redundant dummy interactions)
def is_valid_feature(feature):
    terms = feature.split()  
    for term in terms:
        if '^' in term:
            var, exp = term.split('^')
            if var in ['Downtown', 'Workday', 'Center', 'High'] and int(exp) > 1:
                return False 
    return True

# Filter valid features
valid_feature_indices = [i for i, feature in enumerate(feature_names) if is_valid_feature(feature)]
filtered_features = [feature_names[i] for i in valid_feature_indices]
X_poly_filtered = X_poly[:, valid_feature_indices]

# Standardize features
scaler = StandardScaler()
X_poly_scaled = scaler.fit_transform(X_poly_filtered)

# Ridge Regression with the best alpha determined from Part 1
best_alpha = 0.019306977288832496  
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_poly_scaled, y)

# Predictions and Metrics
y_pred_full = ridge_best.predict(X_poly_scaled)
mse_full = mean_squared_error(y, y_pred_full)
r2_full = r2_score(y, y_pred_full)

# Cross-Validation to Get Test MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
cv_mse_scores = cross_val_score(ridge_best, X_poly_scaled, y, scoring=mse_scorer, cv=5)
cv_test_mse = -np.mean(cv_mse_scores) 

# AIC and BIC Calculations
n_full = X.shape[0]
k_full = X_poly_scaled.shape[1] + 1  # Number of parameters
aic_full = n_full * np.log(mse_full) + 2 * k_full
bic_full = n_full * np.log(mse_full) + k_full * np.log(n_full)

# Generate Summary
residuals = y - y_pred_full
residual_var = np.var(residuals, ddof=1)
ridge_penalty_matrix = best_alpha * np.eye(X_poly_scaled.shape[1])
cov_matrix = np.linalg.inv(X_poly_scaled.T @ X_poly_scaled + ridge_penalty_matrix)
standard_errors = np.sqrt(residual_var * cov_matrix.diagonal())

summary_df = pd.DataFrame({
    "Feature": ["Intercept"] + filtered_features,
    "Coefficient": [ridge_best.intercept_] + ridge_best.coef_.tolist(),
    "Std Error": [np.nan] + standard_errors.tolist()
})

# Sort by the absolute value of coefficients in descending order
summary_df['Abs Coefficient'] = summary_df['Coefficient'].abs()
summary_df = summary_df.sort_values(by="Abs Coefficient", ascending=False).drop(columns="Abs Coefficient")

# Print Final Model Summary
print("\nModel Summary:")
print("=====================================")
print(f"Best Alpha: {best_alpha}")
print(f"R-squared: {r2_full:.4f}")
print(f"Mean Squared Error (MSE): {mse_full:.4f}")
print(f"Cross-Validation Test MSE: {cv_test_mse:.4f}")
print(f"AIC: {aic_full:.4f}")
print(f"BIC: {bic_full:.4f}")
print("=====================================")
print(summary_df.to_string(index=False))


Model Summary:
Best Alpha: 0.019306977288832496
R-squared: 0.9996
Mean Squared Error (MSE): 0.2474
Cross-Validation Test MSE: 0.2487
AIC: -30613.3491
BIC: -30165.4164
                Feature  Coefficient  Std Error
              Intercept    54.652818        NaN
                Shops^2     6.787656   3.084617
                  Shops     6.592232   1.211432
       Shops^2 Downtown     6.447716   2.948906
   Shops Workday Center    -4.768004   0.313408
         Shops Downtown     4.648742   2.851448
                Shops^3     4.445164   1.796655
           Shops Center     4.000660   1.097849
                   ATMs    -3.702765   0.230480
           Shops^2 ATMs    -1.293415   2.053351
     Shops ATMs Workday     1.236881   0.989502
    Shops Downtown High    -1.222662   2.755235
    Shops ATMs Downtown     1.212712   2.931315
               Downtown    -1.168996   1.638568
                 Center     1.016487   0.125169
  ATMs Downtown Workday    -0.995535   0.909528
                